In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from exod.utils.path import data_results
import pandas as pd
from glob import glob
import re

from astropy.visualization import hist
from astropy.table import Table
from astropy.coordinates import SkyCoord, match_coordinates_sky
import astropy.units as u

pd.set_option('display.max_rows', 20)

In [ ]:
# Observations to Exclude (No regions, previous results)
exclude = ['0865470301',
           '0865011601',
           '0872393301',
           '0872790301',
           '0870990201',
           '0891010101',
           '0865320201',
           '0872790501',
           '0891801301',
           '0891802301',
           '0861450301',
           '0891802501',
           '0870930301',
           '0870400101',
           '0862770501',
           '0872390501',
           '0865050301',
           '0865011801',
           '0872790201',
           '0882870201',
           '0871590201',
           '0861680101',
           '0862990201',
           '0862770201',
           '0865011701',
           '0884550101',
           '0865380101',
           '0882160401',
           '0865380301',
           '0872790601',
           '0862990301',
           '0870930501',
           '0865380201',
           '0881420201',
           '0872790401',
           '0881420301',
           '0872391301',
           '0871191001',
           '0870930401',
           '0861880101',
           '0864340101',
           '0865050701',
           '0872790101',
           '0871591801',
           '0872392901',
           '0865470201',
           '0891804001',
           '0882870101',
           '0881420401',
           '0860190301',
           '0870210101',
           '0891802601',
           '0871591301',
           '0870880101',
           '0870930101',
           '0864430201',
           '0891804201',
           '0862090801',
           '0911990401',
           '0865011401',
           '0871190101',
           '0890660101',
           '0861840101',
           '0865011301',
           '0865011001',
           '0882160601',
           '0865010101',
           '0890650101',
           '0872392801',
           '0861680201',
           '0872390801',
           '0865011101',
           '0870931001',
           '0861172301',
           '0891010201',
           '0865011501',
           '0865380401',
           '0891802401',
           '0871590701',
           '0861360101',
           '0862770101',
           '0865011201',
           '0891801901',
           '0882870601']

In [ ]:
# Simulation List
df_simlist = pd.read_csv('../data/results_combined/t_25s_5k_obs/EXOD_simlist_2024-02-16_15-29-21.csv')
df_simlist_run = df_simlist[df_simlist['status'] == 'Run']
df_simlist

In [ ]:
# Detector Info
dfs = []
regex = re.compile(r'\d{10}')

for f in glob('../data/results_combined/t_25s_5k_obs/*/*detector_info.csv'):
    obsid = regex.findall(f)[0]
    if obsid in exclude:
        continue
    df = pd.read_csv(f, index_col=0, header=None, names=['Metric', 'Value'])
    df = df.T  
    df['obsid'] = obsid
    dfs.append(df)

df_detector = pd.concat(dfs, ignore_index=True)
df_detector


In [ ]:
# Event List Info
dfs = []
for f in glob('../data/results_combined/t_25s_5k_obs/*/*evt_info.csv'):
    obsid = regex.findall(f)[0]
    if obsid in exclude:
        continue
    df = pd.read_csv(f, index_col=0, header=None, names=['Metric', 'Value'])
    df = df.T  
    df['obsid'] = obsid
    dfs.append(df)

df_evt = pd.concat(dfs, ignore_index=True)
df_evt


In [ ]:
# Data Loader Info
dfs = []

for f in glob('../data/results_combined/t_25s_5k_obs/*/*dl_info.csv'):
    obsid = regex.findall(f)[0]
    if obsid in exclude:
        continue
    df = pd.read_csv(f, index_col=0, header=None, names=['Metric', 'Value'])
    df = df.T  
    df['obsid'] = obsid
    dfs.append(df)

df_dl = pd.concat(dfs, ignore_index=True)
df_dl


In [ ]:
# Data Cube Info
dfs = []
regex = re.compile(r'\d{10}')
exclude = []  # Add your exclusion list here

for f in glob('../data/results_combined/t_25s_5k_obs/*/*data_cube_info.csv'):
    obsid = regex.findall(f)[0]
    if obsid in exclude:
        continue
    df = pd.read_csv(f, index_col=0, header=None, names=['Metric', 'Value'])
    df = df.T  
    df['obsid'] = obsid
    dfs.append(df)

df_data_cube = pd.concat(dfs, ignore_index=True)
df_data_cube


In [ ]:
# Detected Regions
dfs = []
for f in glob('../data/results_combined/t_25s_5k_obs/*/*regions.csv'):
    
    obsid = regex.findall(f)[0]
    if obsid in exclude:
        continue
    if '_regions' in f:
        continue
    df = pd.read_csv(f)
    df['obsid'] = obsid
    #df = df.T  
    #df['obsid'] = obsid
    dfs.append(df)

df_regions = pd.concat(dfs, ignore_index=True)
df_regions


In [ ]:
# Date Range: 2013-05-05 - 2022-10-11
# Obs run: 3625 / 5000

# Start: 2024-02-16 15:29:21,373
# End:   2024-02-19 03:46:37,864
# Time Taken: 60 hours (3617 min) 
# 3617 / 5000 = 0.72m/obs (over all obs)
# 3617 / 3625 = 1m/obs    (successful obs)

In [ ]:
# All Regions Plot
sky_coords = SkyCoord(ra=df_regions['ra_deg'].values, dec=df_regions['dec_deg'].values, unit='deg', frame='icrs')
sky_coords

ra_rad = sky_coords.ra.wrap_at(180 * u.deg).radian
dec_rad = sky_coords.dec.radian

# Plotting with Aitoff projection
plt.figure(figsize=(15, 10))
plt.subplot(111, projection='aitoff')

plt.title(f'Detected Regions by EXOD: {len(sky_coords)}')

galactic_coords = sky_coords.galactic
galactic_center = SkyCoord(0, 0, unit="deg", frame="galactic")

# Scatter plot the Galactic coordinates
I_log = np.log10(df_regions['intensity_mean'])
m = plt.scatter(galactic_coords.l.wrap_at(180 * u.deg).radian,
            galactic_coords.b.radian,
            marker='s', label='Detected Regions', s=1.0, c=I_log, cmap='hot')

#plt.scatter(0, 0, marker='+', color='Blue', label='Galactic Center')
plt.colorbar(m, shrink=1.0, orientation='horizontal', label='Log Intensity')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
df_data_cube['memory_mb'].astype('float').plot(figsize=(4,3),kind='hist', bins=100, xlabel='Data Cube Memory (MB)')

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,3))
ax[0].hist(np.array(df_evt['exposure'], dtype='float'), bins=50)
ax[1].hist(np.log10(np.array(df_evt['N_events'], dtype='int')), bins=50)
ax[1].set_xlabel(r'$10^{N}$ Events')
ax[0].set_xlabel('Exposure')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5,5))
ax.hist(np.array(df_evt['mean_rate'], dtype='float'), bins=100)
ax.set_xlabel(r'Mean Rate (Events/Exposure)')
plt.show()

In [ ]:
# Crossmatching with XMM catalogue For column descriptions see:
# http://xmmssc.irap.omp.eu/Catalogue/4XMM-DR13/4XMM-DR13_Catalogue_User_Guide.html#Catalogue
# Basic energy bands:
# 1 = 0.2 - 0.5 keV
# 2 = 0.5 - 1.0 keV
# 3 = 1.0 - 2.0 keV
# 4 = 2.0 - 4.5 keV
# 5 = 4.5 - 12.0 keV
# Broad energy bands: 	
# 6 = 0.2 - 2.0  keV | soft band, no images made
# 7 = 2.0 - 12.0 keV | hard band, no images made
# 8 = 0.2 - 12.0 keV | total band
# 9 = 0.5 - 4.5  keV | XID band

tab_xmm = Table.read('../data/util/4XMM_slim_DR13cat_v1.0.fits')
tab_xmm

In [ ]:
skycoord_xmm = SkyCoord(ra=tab_xmm['SC_RA'], dec=tab_xmm['SC_DEC'], unit=u.deg)
skycoord_xmm

In [ ]:
# Find nearest matches
coord1 = sky_coords
coord2 = skycoord_xmm
idx, sep2d, d3d = match_coordinates_sky(coord1, coord2)

# Calculate the differences in RA and Dec for the nearest matches
ra_diff  = coord1.ra  - coord2[idx].ra
dec_diff = coord1.dec - coord2[idx].dec

# Get subset for below specifed distance
dist_arcsec  = 20
mask_sub     = sep2d.arcsec < dist_arcsec
ra_diff_sub  = ra_diff[mask_sub]
dec_diff_sub = dec_diff[mask_sub]

fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].scatter(ra_diff.arcsec, dec_diff.arcsec,  s=1, color='black')
ax[0].set_title(f'Offset, all sources ({len(ra_diff)})')
ax[0].set_xlabel('Ra offset (arcsec)')
ax[0].set_ylabel('Dec offset  (arcsec)')


ax[1].set_title(f'Offset, sources < {dist_arcsec}" ({len(ra_diff_sub)})')
ax[1].scatter(ra_diff_sub.arcsec, dec_diff_sub.arcsec,  s=1, color='black')
ax[1].set_xlabel('Ra offset (arcsec)')
ax[1].set_ylabel('Dec offset  (arcsec)')
#ax[1].set_xlim(-30,30)
#ax[1].set_ylim(-30,30)


fig, ax = plt.subplots(1,2, figsize=(10,3))
ax[0].hist(ra_diff_sub.arcsec, bins=20, color='black')
ax[0].set_xlabel('Ra offset (arcsec)')
ax[1].hist(dec_diff_sub.arcsec, bins=20, color='black', orientation='horizontal')
ax[1].set_xlabel('Dec offset (arcsec)')



In [ ]:
cmatch = sky_coords.match_to_catalog_sky(skycoord_xmm)
tab_cmatch = Table(cmatch)
tab_cmatch.rename_columns(names=tab_cmatch.colnames, new_names=['idx', 'sep2d', 'dist3d'])
tab_cmatch['sep2d_arcsec'] = tab_cmatch['sep2d'].to(u.arcsec)
tab_cmatch['idx_orig'] = np.arange(len(tab_cmatch))
tab_cmatch

In [ ]:
tab_cmatch_good = tab_cmatch[tab_cmatch['sep2d_arcsec'] <= 20]
tab_cmatch_none = tab_cmatch[tab_cmatch['sep2d_arcsec'] > 20]
tab_xmm_good = tab_xmm[tab_cmatch_good['idx']]
tab_xmm_none = tab_xmm[tab_cmatch_none['idx']]

In [ ]:
# Crossmatch Plot
sc_good = SkyCoord(ra=tab_xmm_good['SC_RA'], dec=tab_xmm_good['SC_DEC'], unit='deg', frame='icrs')
sc_none = sky_coords[tab_cmatch_none['idx_orig']]

def sc_to_gal(sc):
    ra_rad = sc.ra.wrap_at(180 * u.deg).radian
    dec_rad = sc.dec.radian
    return sc.galactic
    
sc_gal_good = sc_to_gal(sc_good)
sc_gal_none = sc_to_gal(sc_none)

# Plotting with Aitoff projection
plt.figure(figsize=(15, 10))
plt.subplot(111, projection='aitoff')
plt.title(f'Detected Regions by EXOD: {len(sky_coords)}')
plt.scatter(sc_gal_good.l.wrap_at(180 * u.deg).radian, sc_gal_good.b.radian, marker='.', label=f'Counterpart Sources {len(tab_xmm_good)}', s=1.0, color='blue')
plt.scatter(sc_gal_none.l.wrap_at(180 * u.deg).radian, sc_gal_none.b.radian, marker='.', label=f'No Counterparts {len(tab_xmm_none)}', s=1.0, color='red')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Plot sources with large seperations
mask_sub = abs(dec_diff.arcsec) > 1000

# All Regions Plot
sky_coords = coord1[mask_sub]
ra_rad = sky_coords.ra.wrap_at(180 * u.deg).radian
dec_rad = sky_coords.dec.radian

# Plotting with Aitoff projection
plt.figure(figsize=(15, 10))
plt.subplot(111, projection='aitoff')
plt.title(f'Sources with abs(Dec) > 1000 arcsec')
galactic_coords = sky_coords.galactic
galactic_center = SkyCoord(0, 0, unit="deg", frame="galactic")

# Scatter plot the Galactic coordinates

m = plt.scatter(galactic_coords.l.wrap_at(180 * u.deg).radian,
            galactic_coords.b.radian, marker='s', label='Detected Regions', s=1.0)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
N_src  = len(tab_cmatch)
N_good = len(tab_cmatch_good)

hist(tab_cmatch_good['sep2d_arcsec'], bins='knuth', histtype='stepfilled', ec='k', fc='#AAAAAA')
plt.title(rf'Distribution of {N_good}/{N_src} regions with with XMM counterparts $\leq$ 20"')
plt.xlabel('Seperation (arcseconds)')

mu  = np.mean(tab_cmatch_good['sep2d_arcsec'])
std = np.std(tab_cmatch_good['sep2d_arcsec'])
plt.axvline(mu, color='red', label=fr'mean={mu:.2f}$\pm${std:.2f}')
plt.axvline(mu+std, color='green')
plt.axvline(mu-std, color='green')
plt.legend()
plt.show()

In [ ]:
N_src  = len(tab_cmatch)
N_good = len(tab_cmatch_none)

hist(tab_cmatch_none['sep2d_arcsec'], bins='knuth', histtype='stepfilled', ec='k', fc='#AAAAAA')
plt.title(rf'Distribution of {N_good}/{N_src} regions with with XMM counterparts > 20"')
plt.xlabel('Seperation (arcseconds)')
plt.yscale('log')
plt.show()

In [ ]:
# Count Rate vs Chi^2 Prob
plt.figure(figsize=(6,6))
plt.scatter(tab_xmm_good['SC_CHI2PROB'], tab_xmm_good['SC_EP_1_FLUX'], marker='.', s=1, color='black', label='SC_EP_1_FLUX')
plt.scatter(tab_xmm_good['SC_CHI2PROB'], tab_xmm_good['SC_EP_5_FLUX'], marker='.', s=1, color='blue', label='SC_EP_5_FLUX')
plt.xlim(xmin=0)
plt.yscale('log')
plt.xlabel('SC_CHI2PROB')
plt.ylabel('FLUX')
plt.legend()
plt.show()

In [ ]:
# Count Rate vs F_var
plt.figure(figsize=(6,6))
plt.scatter(tab_xmm_good['SC_HR1'], tab_xmm_good['SC_EP_1_FLUX'], marker='.', s=1, color='black', label='SC_EP_1_FLUX')
plt.scatter(tab_xmm_good['SC_HR1'], tab_xmm_good['SC_EP_5_FLUX'], marker='.', s=1, color='blue', label='SC_EP_5_FLUX')
plt.xlim(xmin=0)
plt.yscale('log')
#plt.xscale('log')
plt.xlabel('N_DETECTIONS')
plt.ylabel('FLUX')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
sub = tab_xmm_good[(tab_xmm_good['SC_CHI2PROB'] > 0.8) & (tab_xmm_good['SC_HR1'] > 0.75)]
plt.scatter(tab_xmm_good['SC_CHI2PROB'], tab_xmm_good['SC_HR1'], marker='.', s=1, color='black')
plt.scatter(sub['SC_CHI2PROB'], sub['SC_HR1'], marker='.', s=20, color='red')
plt.xlim(xmin=0)
#plt.yscale('log')
plt.xlabel('SC_CHI2PROB')
plt.ylabel('SC_HR1')
plt.show()